## The Cython language

* The Cython language is a superset of the Python language -- nearly any Python code is valid Cython code, with very few edge cases.

## Part 1: brief tour of Cython types

* Dynamically typed variables are simply Cython `object` types; treated as dynamically typed and reference counted `PyObject` pointer objects at the C level.
* Cython introduces the `cdef` keyword to declare a C-level construct.
* Cython has other constructs--almost all beginning with `cdef`--to declare more complex C-only constructs.

## Simple types

In [12]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [13]:
%%cython

# untyped objects are simply Python declarations like we know and love...
o = 1
print(o)

# and can be reassigned to a different type...
o = 'a'
print(o)

## Integral types

In [14]:
%%cython

# The panoply of C integer types...
cdef:
    int i = 0
    unsigned long j = 1
    signed short k = -3
    long long ll = 1LL

print(i, j, k, ll)

### Cython enforces static typing (either compile time or runtime, depending)

In [15]:
%%cython

cdef int i = 0

# ...

i = 'qwerty'


Error compiling Cython file:
------------------------------------------------------------
...

cdef int i = 0

# ...

i = 'qwerty'
   ^
------------------------------------------------------------

/Users/kusmith/.ipython/cython/_cython_magic_913bc80a7df7ded79c2587d07ae49234.pyx:6:4: Unicode literals do not support coercion to C types other than Py_UNICODE/Py_UCS4 (for characters) or Py_UNICODE* (for strings).


## Floating point types

In [16]:
%%cython

cdef:
    float a = 1.0
    double b = -1.0
    long double c = 1e100
print(a, b, c)

## Complex types

In [17]:
%%cython

cdef:
    float complex fc = 1+1j
    double complex dc = 1+1j
    long double complex ldc = 1+1j

print(fc, dc, ldc)
print(fc.real, dc.imag, ldc.conjugate())

## String types

* Note that string types in Python are complicated by Py2 / Py3 differences.
* This tutorial is entirely in Python 3, so we do not have to go into the minutiae here, which we declare to be out of scope.
* See [the fine manual](http://cython.readthedocs.io/en/latest/src/tutorial/strings.html#python-string-types-in-cython-code) or [other documentation](http://shop.oreilly.com/product/0636920033431.do) for more details.

### Statically declaring Python string types in Cython

In [18]:
%%cython

cdef str s = "asdf"
cdef bytes b = b"jkl;"
print(s, b)

##### Cython allows static declaration of Python types!

### C "string"s

In [19]:
%%cython

bb = b"asdf"
cdef char *buf = bb
print(bb, buf)

#### Pop quiz:

* What can we infer from the (runtime) warning generated here?
* When working with C-level strings, what must we keep in mind to maintain Python guarantees?

It's often best to just work with `str` and `bytes` objects, rather than acquiring the underlying buffer pointer.

In [20]:
%%cython

# Why is this very, very bad?
a = b'a'
b = b'b'
cdef char *c = a + b

# ...and how can we address it?

# Again, it's best to avoid using char *'s 
# unless absolutely necessary -- keep your
# Strings Python-level strings, and all will
# be copacetic.


Error compiling Cython file:
------------------------------------------------------------
...

# Why is this very, very bad?
a = b'a'
b = b'b'
cdef char *c = a + b
    ^
------------------------------------------------------------

/Users/kusmith/.ipython/cython/_cython_magic_b4c4f60099ad1d9f809d701b234494c4.pyx:5:5: Storing unsafe C derivative of temporary Python reference


## Other statically-declarable Python types

In [21]:
%%cython

import datetime
cimport cpython.datetime # We'll cover the `cimport` keyword later

import array
cimport cpython.array

cdef:
    list lst = [1]
    dict dd = {'a': 'b'}
    set ss = set([1])
    frozenset fs = frozenset([1])
    cpython.datetime.datetime dt = datetime.datetime.now()
    cpython.array.array aa = array.array('i', [1, 2, 3])
    
print(lst, dd, ss, fs, dt, aa)

[1] {'a': 'b'} {1} frozenset({1}) 2017-06-19 20:18:53.914916 array('i', [1, 2, 3])


### Nota Bene:

* Everything on the lhs of the declaration is a C-level, statically typed entity.
* Everything on the RHS of the declaration is a Python object of the corresponding type.
* Cython enforces static type checking for these objects.

## Declaring and manipulating pointers

**NOTE** Only do this if you have to -- fortunately we rarely have to drop to using or manipulating pointers in Cython unless we're interfacing with a C library.

In [22]:
%%cython

cdef:
    int *a = NULL
    int b = 42

# Point `a` at `b`
a = &b

# Dereference `a` -- Note the use of `a[0]` -- we don't use `*a` to dereference!
print(a[0], b)

# Modify b via a:
a[0] = 137

# Confirm that both refer to same value:
print(a[0], b)

42 42
137 137


## Structs, Unions

* Typically used to interface with C code; not typically used for Python-centric workflows.

In [25]:
%%cython

cdef struct s_t:
    int i, j, k
    float u, v, w
    
cdef s_t s